In [51]:
sum_train_score = 0
sum_test_score = 0
iteraciones = 10

FunH = 'tanh'   # identity logistic tanh relu
ocultas = (13)
alfa = 0.01
MAX_ITE = 2000

import pandas as pd      # para trabajar con archivos de datos csv, excel, etc: https://pandas.pydata.org/docs/getting_started/tutorials.html
import chardet           # para detectar la codificación de texto en archivos
import numpy as np

from sklearn import preprocessing, model_selection
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot as plt

nombre_archivo = '../Datos/Zoo.csv' # archivo de hojas

#-- detectando la codificación de caracteres usada ----
with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

# recupera el archivo en un objeto dataframe de pandas utilizando la codificación detectada
df = pd.read_csv(nombre_archivo, encoding=result['encoding'])

# %% separa atributos y clases
X_raw = np.array(df.iloc[:,1:-1])  # recupera todas las columnas salvo la primera (es la clase)
Y_raw = np.array(df.iloc[:,-1])    # recupera solo la última columna (es la clase)

# Binarizador para convertir el nombre de la clase en one hot encoding
binarizer = preprocessing.LabelBinarizer()

# Binariza cada clase como una combinación de un 1 y 0s
Y_raw = binarizer.fit_transform(Y_raw)
# print('Las clases del dataset son :', binarizer.classes_)



In [52]:
for i in range(iteraciones):

    # Separa ejemplos para enternamiento y testeo
    TEST_SIZE = 0.3 # proporcion entre testeo entre entrenamiento y testeo
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X_raw, Y_raw, test_size=TEST_SIZE)#, random_state=42)

    # print('\nDatos de Entrenamiento: %d   Datos de Testeo: %d' % (len(Y_train), len(Y_test) ))

    # Escala los atributos de los ejemplo
    scaler = preprocessing.StandardScaler()
    #scaler = preprocessing.MinMaxScaler()
    X_train  = scaler.fit_transform( X_train )
    X_test   = scaler.transform( X_test )

    modelo = MLPClassifier(max_iter=MAX_ITE, hidden_layer_sizes=ocultas, alpha=alfa,
                           activation=FunH,
#                            tol=0.0002, #esto es la tolerancia al fallo, todavía no lo vimos
                           verbose=False).fit(X_train, Y_train)
    #SKLEARN ELIGE entropía cruzada PARA USAR COMO FUNCIÓN DEL ERROR PARA MLPCLASSIFIER

    #  ########### Medición del entrenamiento ######################
    Y_pred = modelo.predict(X_train)
    score = modelo.score(X_train, Y_train)

    # "invierte" la transformacion binaria para obtener los nombres de las clases
    Y_it = binarizer.inverse_transform(Y_train)
    Y_pred_it = binarizer.inverse_transform(Y_pred)

    # calculo manual del accuracy
    # print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )
    # print('      Score: %6.2f%%' % (score) )
    print(f"train score iter {i+1}: {score}")
    sum_train_score += score
    # plt.plot(modelo.loss_curve_)
    
    #  ########### Medición del testeo ######################
    Y_pred = modelo.predict(X_test)
    score = modelo.score(X_test, Y_test)

    # "invierte" la transformacion binaria para obtener los nombres de las clases
    Y_it = binarizer.inverse_transform(Y_test)
    Y_pred_it = binarizer.inverse_transform(Y_pred)

    # calculo manual del accuracy
    # print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )
    # print('   R2 Score: %6.2f%%' % (score) )
    print(f"test score iter {i+1}: {score}")
    sum_test_score += score

train score iter 1: 1.0
test score iter 1: 1.0
train score iter 2: 1.0
test score iter 2: 0.8064516129032258
train score iter 3: 1.0
test score iter 3: 0.9354838709677419
train score iter 4: 1.0
test score iter 4: 0.9354838709677419
train score iter 5: 1.0
test score iter 5: 0.9032258064516129
train score iter 6: 1.0
test score iter 6: 0.9354838709677419
train score iter 7: 1.0
test score iter 7: 0.9354838709677419


C:\Users\Mateo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


train score iter 8: 1.0
test score iter 8: 0.9032258064516129


C:\Users\Mateo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


train score iter 9: 1.0
test score iter 9: 1.0
train score iter 10: 1.0
test score iter 10: 0.967741935483871


C:\Users\Mateo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [53]:
print('Avg train Score: %6.2f%%' % (float(sum_train_score)/iteraciones ))
print('Avg test Score: %6.2f%%' % (float(sum_test_score)/iteraciones ))

Avg train Score:   1.00%
Avg test Score:   0.93%


# Resultados: 
## al principio usaba un max_iteraciones de 500, pero el modelo nunca convergía
|capas ocultas|avg train score|avg test score|
|---|---|---|
|4|0.48|0.52|
|8|0.83|0.76|
## por este motivo empecé a usar un max iterations de 2000
|capas ocultas|avg train score|avg test score|
|---|---|---|
|12|1|0.87|
## como no estamos analizando imágenes pensé que era mejor dejar de usar relu para probar con tanh, la cual nos dio los siguientes resultados
|capas ocultas|avg train score|avg test score|
|---|---|---|
|12|1|0.90|
|15|1|0.94|
|13|1|0.93|